In [13]:
import numpy as np
import pandas as pd
import random
%run DataAnalysis.ipynb


class MonteCarlo():
    def __init__(self):

        
    
    # create dictionary mapping channel numbers to a tuple containing coordinates (x,y,z)
    def load_coords(self):

        coords = {}

        with open(pwd + "/data/detector_positions.txt", 'r') as f:
            for line in f:
                data = line.split(',')

                if int(data[0]) < 1000:
                    coords[int(data[0])] = (float(data[1]), float(data[2]), float(data[3]))

        return coords
    
    
    def generate_tracks(self, num=5):
        
        tracks = []
        
        for track in range(num):
            a = [random.uniform(-350,350), random.uniform(-350,350), random.uniform(-350,350)]
            b = [random.uniform(-350,350), random.uniform(-350,350), random.uniform(-350,350)]
            
            tracks.append(np.array([a,b]))
            
        return tracks
            
    
    def lineplanecollision(self, planeNormal, planePoint, rayDirection, rayPoint, epsilon=1e-6):

        ndotu = planeNormal.dot(rayDirection)
        if abs(ndotu) < epsilon:
            return None

        t = -planeNormal.dot(rayPoint - planePoint) / ndotu

        return rayPoint + t * rayDirection

    
    def linecubecollision(self, cubeCenter, cubeLength, rayDirection, rayPoint, epsilon=1e-6):

        cubeCollisions = []

        halfLength = cubeLength / 2.0

        directions = np.array([
            [0,0,halfLength], #up
            [0,halfLength,0], #front
            [halfLength,0,0], #right
        ])

        planeCollisions = []
        for i in range(6):
            if i >= 3:
                faceNormal = -directions[i%3] # to get down, back, left
            else:
                faceNormal = directions[i]

            facePoint = cubeCenter + faceNormal

            collision = self.lineplanecollision(faceNormal, facePoint, rayDirection, rayPoint)
            if collision is not None:
                planeCollisions.append(collision)

        #check if intersection is outside cube
        for collision in planeCollisions:

            inside = True
            for i in range(3):
                if collision[i] > (cubeCenter[i] + halfLength + epsilon) or collision[i] < (cubeCenter[i] - halfLength - epsilon):
                    inside = False

            if inside:
                cubeCollisions.append(collision)

        return cubeCollisions
    
    
    def channelcollisions(self, linepoints, epsilon=1e-6):
                
        rayDirection = linepoints[1] - linepoints[0]
        rayPoint = linepoints[0]
        cubeLength = 50
        
        coords = self.load_coords()
        
        #start = time.time()
        
        hit_channels = []
        for channel in range(1,len(coords)+1):
            cubeCenter = coords[channel]

            #check if cubeCenter is within range of line
            CP = cubeCenter - linepoints[0]
            distance_to_line = np.abs(np.linalg.norm(cross(CP,rayDirection)) / np.linalg.norm(rayDirection))

            #print(distance_to_line)

            if distance_to_line < cubeLength/2*np.sqrt(3) + epsilon:

                collision = self.linecubecollision(cubeCenter, cubeLength, rayDirection, rayPoint)
                if len(collision) == 2:
                    hit_channels.append(channel)
        
        #print(start - time.time())
        
                    
        return hit_channels

In [14]:
mc = MonteCarlo()

tracks = mc.generate_tracks()
len(tracks)


#tracks[0]
mc.channelcollisions(tracks[0])

AttributeError: 'MonteCarlo' object has no attribute 'load_coords'